In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from time import sleep
import json
import datetime

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from time import sleep
import json
import datetime


# edit these three variables
user = 'realdonaldtrump'
start = datetime.datetime(2016, 1, 1)  # year, month, day
end = datetime.datetime(2019, 10, 21)  # year, month, day

# only edit these if you're having problems
delay = 1  # time to wait on each page load before reading the page
driver = webdriver.Safari()  # options are Chrome() Firefox() Safari()


# don't mess with this stuff
twitter_ids_filename = 'all_ids.json'
days = (end - start).days + 1
id_selector = '.time a.tweet-timestamp'
tweet_selector = 'li.js-stream-item'
user = user.lower()
ids = []

def format_day(date):
    day = '0' + str(date.day) if len(str(date.day)) == 1 else str(date.day)
    month = '0' + str(date.month) if len(str(date.month)) == 1 else str(date.month)
    year = str(date.year)
    return '-'.join([year, month, day])

def form_url(since, until):
    p1 = 'https://twitter.com/search?f=tweets&vertical=default&q=from%3A'
    p2 =  user + '%20since%3A' + since + '%20until%3A' + until + 'include%3Aretweets&src=typd'
    return p1 + p2

def increment_day(date, i):
    return date + datetime.timedelta(days=i)

for day in range(days):
    d1 = format_day(increment_day(start, 0))
    d2 = format_day(increment_day(start, 1))
    url = form_url(d1, d2)
    print(url)
    print(d1)
    driver.get(url)
    sleep(delay)

    try:
        found_tweets = driver.find_elements_by_css_selector(tweet_selector)
        increment = 10

        while len(found_tweets) >= increment:
            print('scrolling down to load more tweets')
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(delay)
            found_tweets = driver.find_elements_by_css_selector(tweet_selector)
            increment += 10

        print('{} tweets found, {} total'.format(len(found_tweets), len(ids)))

        for tweet in found_tweets:
            try:
                id = tweet.find_element_by_css_selector(id_selector).get_attribute('href').split('/')[-1]
                ids.append(id)
            except StaleElementReferenceException as e:
                print('lost element reference', tweet)

    except NoSuchElementException:
        print('no tweets on this day')

    start = increment_day(start, 1)

    
try:
    with open(twitter_ids_filename) as f:
        all_ids = ids + json.load(f)
        data_to_write = list(set(all_ids))
        print('tweets found on this scrape: ', len(ids))
        print('total tweet count: ', len(data_to_write))
except FileNotFoundError:
    with open(twitter_ids_filename, 'w') as f:
        all_ids = ids
        data_to_write = list(set(all_ids))
        print('tweets found on this scrape: ', len(ids))
        print('total tweet count: ', len(data_to_write))

with open(twitter_ids_filename, 'w') as outfile:
    json.dump(data_to_write, outfile)

print('all done here')
driver.close()


https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2016-01-01%20until%3A2016-01-02include%3Aretweets&src=typd
2016-01-01
scrolling down to load more tweets
14 tweets found, 0 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2016-01-02%20until%3A2016-01-03include%3Aretweets&src=typd
2016-01-02
scrolling down to load more tweets
11 tweets found, 14 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2016-01-03%20until%3A2016-01-04include%3Aretweets&src=typd
2016-01-03
scrolling down to load more tweets
18 tweets found, 25 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2016-01-04%20until%3A2016-01-05include%3Aretweets&src=typd
2016-01-04
4 tweets found, 43 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2016-01-05%20until%3A2016-01-06include%3Aretweets&src=typd
2016-01-05
scr

19 tweets found, 528 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2016-02-07%20until%3A2016-02-08include%3Aretweets&src=typd
2016-02-07
scrolling down to load more tweets
scrolling down to load more tweets
25 tweets found, 547 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2016-02-08%20until%3A2016-02-09include%3Aretweets&src=typd
2016-02-08
scrolling down to load more tweets
10 tweets found, 572 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2016-02-09%20until%3A2016-02-10include%3Aretweets&src=typd
2016-02-09
scrolling down to load more tweets
15 tweets found, 582 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2016-02-10%20until%3A2016-02-11include%3Aretweets&src=typd
2016-02-10
9 tweets found, 597 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2016-0

KeyboardInterrupt: 

In [4]:
import tweepy
import json
import math
import glob
import csv
import zipfile
import zlib
from tweepy import TweepError
from time import sleep

# CHANGE THIS TO THE USER YOU WANT
user = 'realdonaldtrump'


with open('api_keys.json') as f:
    keys = json.load(f)

auth = tweepy.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth)
user = user.lower()
output_file = '{}.json'.format(user)
output_file_short = '{}_short.json'.format(user)
compression = zipfile.ZIP_DEFLATED

with open('all_ids.json') as f:
    ids = json.load(f)

print('total ids: {}'.format(len(ids)))

all_data = []
start = 0
end = 100
limit = len(ids)
i = math.ceil(limit / 100)

for go in range(i):
    print('currently getting {} - {}'.format(start, end))
    sleep(6)  # needed to prevent hitting API rate limit
    id_batch = ids[start:end]
    start += 100
    end += 100
    tweets = api.statuses_lookup(id_batch)
    for tweet in tweets:
        all_data.append(dict(tweet._json))

print('metadata collection complete')
print('creating master json file')
with open(output_file, 'w') as outfile:
    json.dump(all_data, outfile)

print('creating ziped master json file')
zf = zipfile.ZipFile('{}.zip'.format(user), mode='w')
zf.write(output_file, compress_type=compression)
zf.close()

results = []

def is_retweet(entry):
    return 'retweeted_status' in entry.keys()

def get_source(entry):
    if '<' in entry["source"]:
        return entry["source"].split('>')[1].split('<')[0]
    else:
        return entry["source"]

with open(output_file) as json_data:
    data = json.load(json_data)
    for entry in data:
        t = {
            "created_at": entry["created_at"],
            "text": entry["text"],
            "in_reply_to_screen_name": entry["in_reply_to_screen_name"],
            "retweet_count": entry["retweet_count"],
            "favorite_count": entry["favorite_count"],
            "source": get_source(entry),
            "id_str": entry["id_str"],
            "is_retweet": is_retweet(entry)
        }
        results.append(t)

print('creating minimized json master file')
with open(output_file_short, 'w') as outfile:
    json.dump(results, outfile)

with open(output_file_short) as master_file:
    data = json.load(master_file)
    fields = ["favorite_count", "source", "text", "in_reply_to_screen_name", "is_retweet", "created_at", "retweet_count", "id_str"]
    print('creating CSV version of minimized json master file')
    f = csv.writer(open('{}21-10-19_trump.csv'.format(user), 'w'))
    f.writerow(fields)
    for x in data:
        f.writerow([x["favorite_count"], x["source"], x["text"], x["in_reply_to_screen_name"], x["is_retweet"], x["created_at"], x["retweet_count"], x["id_str"]])

total ids: 19312
currently getting 0 - 100
currently getting 100 - 200
currently getting 200 - 300
currently getting 300 - 400
currently getting 400 - 500
currently getting 500 - 600
currently getting 600 - 700
currently getting 700 - 800
currently getting 800 - 900
currently getting 900 - 1000
currently getting 1000 - 1100
currently getting 1100 - 1200
currently getting 1200 - 1300
currently getting 1300 - 1400
currently getting 1400 - 1500
currently getting 1500 - 1600
currently getting 1600 - 1700
currently getting 1700 - 1800
currently getting 1800 - 1900
currently getting 1900 - 2000
currently getting 2000 - 2100
currently getting 2100 - 2200
currently getting 2200 - 2300
currently getting 2300 - 2400
currently getting 2400 - 2500
currently getting 2500 - 2600
currently getting 2600 - 2700
currently getting 2700 - 2800
currently getting 2800 - 2900
currently getting 2900 - 3000
currently getting 3000 - 3100
currently getting 3100 - 3200
currently getting 3200 - 3300
currently gett

In [5]:
import pandas as pd

In [6]:
df_new = pd.read_csv('realdonaldtrump21-10-19_trump.csv')

In [7]:
df_new.head()

,favorite_count,source,text,in_reply_to_screen_name,is_retweet,created_at,retweet_count,id_str
0,83531,Twitter for iPhone,I am pleased to inform you that THE BIG FIREWO...,NaN,False,Wed May 08 01:51:39 +0000 2019,16096,1125941300518432771
1,100395,Twitter for iPhone,Do you notice the Fake News Mainstream Media n...,NaN,False,Tue Jan 16 14:19:46 +0000 2018,22480,953270558573154305
2,2863,Twitter for iPhone,@LittleMissFlint Yes … no … maybe …,LittleMissFlint,False,Sun Dec 02 01:55:17 +0000 2018,66,1069047317670424578
3,35967,Twitter for iPhone,Great night in WI. I’m going to fight for ever...,NaN,False,Tue Oct 18 02:07:55 +0000 2016,14301,788199901209452544
4,230495,Twitter for Android,MAKE AMERICA GREAT AGAIN!,NaN,False,Sat Feb 04 14:26:10 +0000 2017,56694,827885966509604865


In [10]:
df_new.created_at = pd.to_datetime(df_new.created_at)

In [13]:
df_new.sort_values('created_at', ascending=False)

,favorite_count,source,text,in_reply_to_screen_name,is_retweet,created_at,retweet_count,id_str
10879,19098,Twitter Media Studio,https://t.co/oSn6amjZo4,NaN,False,2019-10-21 03:54:07+00:00,6654,1186128505316229121
14829,22773,Twitter for iPhone,https://t.co/R3Chppjx6S,NaN,False,2019-10-21 02:57:37+00:00,8055,1186114286642515969
10309,18650,Twitter for iPhone,https://t.co/6sKKniiOfE,NaN,False,2019-10-21 02:53:32+00:00,6375,1186113259780345857
17286,27271,Twitter for iPhone,A Great Democrat Scam! https://t.co/LbkVT7UvXX,NaN,False,2019-10-21 02:07:51+00:00,9322,1186101761368571904
2067,42614,Twitter for iPhone,Adam Schiff is a Corrupt Politician! https://t...,NaN,False,2019-10-21 02:05:14+00:00,13162,1186101103676526592
4604,21491,Twitter for iPhone,Congratulations Barbara! https://t.co/0cYrjL1YOj,NaN,False,2019-10-21 01:57:10+00:00,5265,1186099074371596288
15945,27409,Twitter for iPhone,"“To the Democrats, Impeachment is partisan pol...",NaN,False,2019-10-21 01:48:35+00:00,7098,1186096913520693248
13526,62911,Twitter for iPhone,"So interesting that, when I announced Trump Na...",NaN,False,2019-10-20 22:41:10+00:00,17052,1186049749989892105
10277,51286,Twitter for iPhone,....because their so-called story didn’t come ...,realDonaldTrump,False,2019-10-20 21:45:17+00:00,13032,1186035685163257857
9401,58385,Twitter for iPhone,....fiction to Congress and the American Peopl...,realDonaldTrump,False,2019-10-20 21:45:17+00:00,15989,1186035686396321793


In [18]:
donald_new = df_new[df_new.created_at.dt.year>2016].sort_values('created_at', ascending=False)

In [19]:
donald_new.to_csv('donald_new.csv')